In [3]:
import numpy as np
from qiskit_aer import Aer
from qiskit import QuantumCircuit,transpile
from PIL import Image,ImageDraw,ImageFont
import math
from math import pi

brightness_cache={};

def quantum(inp_path,out_path,block_size,font_size,magnitude,ascii_set,use_color=True):
    try:
        input_image=Image.open(inp_path)
    except FileNotFoundError:
        print("The requested image is not found!")
        return 
    
    input_width,input_height=input_image.size
    rows=input_height//block_size
    cols=input_width//block_size
    input_image2= input_image.resize((cols, rows), Image.Resampling.LANCZOS)
        # Font setup
    font = ImageFont.load_default()
    #output dim
    bbox=font.getbbox("A")
    char_width=bbox[2]-bbox[0]
    char_height=bbox[3]-bbox[1]
    output_width=cols*char_width
    output_height=rows*char_height
    #initializing a new canvas
    output=Image.new("RGB",(output_width,output_height),"black")
    draw=ImageDraw.Draw(output)
    simulator = Aer.get_backend('qasm_simulator')
    def get_brightness(classical_brightness):
        if classical_brightness in brightness_cache:
            return brightness_cache[classical_brightness]
        
        num_qubits=2
        qc=QuantumCircuit(num_qubits,num_qubits)
        theta=(classical_brightness/255.0)*(pi/2)
        qc.ry(theta,0)
        qc.ry(theta,1)

        if(magnitude)>0:
            qc.h(0)
            qc.cry((classical_brightness/255.0)*(pi/4),0,1)

        qc.measure(range(num_qubits),range(num_qubits))
        final=transpile(qc,simulator)
        job=simulator.run(final,shots=1)
        result=job.result()
        counts=result.get_counts (qc)
        
        #now lets convert the count values back to brightness
        if counts:
            res=list(counts.keys())[0]
            #convert from binary to int and then normalize to adjust
            normalization_factor=int(res,2)/(2**num_qubits-1)
            quantum_brightness=(normalization_factor*255)
            #USE LERP TO BLEND: general formula: a * (1 - t) + b * t(by a factor of t) in our case t=magnitude
            blended_brightness=int(classical_brightness*(1-magnitude)+quantum_brightness)
            brightness_cache[classical_brightness]=blended_brightness
            return blended_brightness

#input image processing
#use the get_brightness for each r,g,b values

    '''perceived brightness formulas:
    1)0.2126*R + 0.7152*G + 0.0722*B
    2)0.299*R + 0.587*G + 0.114*B'''
    #Lets use 1

 
    for row in range(rows):
        for col in range (cols):
            if use_color:
                r,g,b=input_image2.convert("RGB").getpixel((col,row))
                r_new=get_brightness(r)
                g_new=get_brightness(g)
                b_new=get_brightness(b)
                fill=(r_new,g_new,b_new)
                brightness=int(0.2126*r+0.7152*g+0.0722*b)
            else:
                gray = int(0.2126 * r + 0.7152 * g + 0.0722 * b)
                brightness=get_brightness(gray)
                brightness = get_brightness(gray)
                fill =(brightness, brightness, brightness)

            #converting brightness into charecters values based on intensity 
            char_id=min(int((brightness/255)*(len(ascii_set)-1)),len(ascii_set)-1)
            ascii_val=ascii_set[char_id] 
            #drawing with ascii chars now
            draw.text((col*char_width,row*char_height),ascii_val,font=font,fill=fill)
    output.save(out_path) 
    print("Your desired image is saved!!")
    return output

if __name__=="__main__":
    block_size=6
    magnitude=0.0000000001
    font_size=10
    inp_path='testing.jpg'
    output_path='resultv2.jpg'
    ascii_set=" .'`^\",:;Il!i><~+_-?][}{1)(|/tfjrxnuvczXYUJCLQ0OZmwqpdbkhao*#MW&8%B@$"     
    use_color=True
    quantum(inp_path,output_path,block_size,font_size,magnitude,ascii_set,use_color=True)


Your desired image is saved!!
